해시태그 컬럼의 데이터값으로 LAD TopicModeling 분석.

In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
from urllib.parse import quote
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import re

In [ ]:
df=pd.read_csv('./관광지정보최종.csv')

In [ ]:
#  해시태그 컬럼에서 불필요한 값을 가진 행 삭제(관광지삭제)
tags_1 = ['의료관광']
list_a_1=df[df['해시태그_sub'].isin(tags_1)]
tags_2 = ['카페']
list_a_2=df[df['해시태그_sub'].isin(tags_2)]

In [2]:
df.to_csv('./result/240122_tag_re.csv', index=False)
topic = pd.read_csv('./result/240122_tag_re.csv')

In [3]:
topic.head(1)

,관광지명,주소,상세정보,해시태그_sub,평균별점,주소_동별,연락처,위도,경도,해시태그,상세페이지_url
0,성산일출봉(UNESCO세계자연유산),제주특별자치도 서귀포시 성산읍 일출로 284-12,성산일출봉은 제주도의 다른 오름들과는 달리 마그마가 물속에서 분출하면서 만들어진 수...,"일출,오름,경관,포토,부모,자연경관,포토스팟,유네스코,무장애관광,공영관광지",별점(5점만점에 5점),성산읍 성산리,(+82) 064-783-0959,33.462234,126.936801,"['#일출', '#오름', '#경관/포토', '#부모', '#자연경관', '#포토스...",https://www.visitjeju.net/kr/detail/view?conte...


In [4]:
topic['해시태그_sub'][0]

'일출,오름,경관,포토,부모,자연경관,포토스팟,유네스코,무장애관광,공영관광지'

In [12]:
web_stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

stopwords = set([])
for i in web_stopwords:
    stopwords.add(i[0])

len(stopwords)


# 불용어 추가
jeju_stopwords = ['제주', '제주도',  '리뷰', '숙소', '여행', '트립', '정말', '동안', '다음', '바로', '근처', '조금', '시간',
                   '호스트', '정도', '방문', '사장', '이용', '다시', '예약', '살이', '살기', '아주', '추천', '하루', '이번',
                   '처음', '달동', '저희', '하나', '사용', '모두', '진짜', '일주일', '보름', '선택', '시작', '한번', '가장', '마음',
                   '제외', '이동', '무엇', '준비', '그냥', '감사', '생각', '느낌', '덕분', '후기', '기회',  '다른', '여기', '우리',
                   '미스터', '미스터멘션', '멘션', '때문', '오늘', '도착', '보고', '마지막', '가지', '이제', '지금', '모습', '위해',
                   '타고', '사실', '포스팅', '주문', '역시', '침대', '거실', '매일', '출발', '소개', '기억', '참고', '계속', '객실',
                   '일차', '이야기', '요즘', '이유', '경우', '일단', '거의', '스텝', '제일', '자주', '어디', '마무리', '아래', '운영',
                   '이상', '의자', '별로', '원래', '중간', '뭔가', '첫째', '체크', '잠시', '언제', '살짝', '해도', '계단', '부분', '냉장고',
                    '침실', '기본', '제공', '일찍', '인테리어', '완전', '추가', '등등', '매우', '가야', '그때', '기간', '마치', '가득', '대부분',
                    '미리', '둘째', '근무', '공간', '스테이', '알파', '엄마', '아빠', '아들', '일정', '계획', '아침',
                     '점심', '저녁', '자리', '두리', '언니', '가격', '건물', '언니', 
                      '숙박', '만원', '가기', '독채','화장실', '욕실', '숙박', '세탁기', '코로나' ,
                     '주방', '도서관', '서울','티비', '트랙','이드','택트','입장','봄봄',
                     '호텔', '펜션', '게스트하우스']
for word in jeju_stopwords:
    stopwords.add(word)

In [6]:
from konlpy.tag import Okt
from collections import Counter
okt = Okt()

In [13]:
def hangul_function(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

In [14]:
temp_list = []
for blog in topic['해시태그_sub']:
    LDA_nous = okt.nouns(hangul_function(blog))
    LDA_nous = [x for x in LDA_nous if len(x) > 1 or x =='봄']  # 한글자 키워드 제거    
    LDA_nous = [word for word in LDA_nous if not word in stopwords]
    temp_list.append(LDA_nous)

In [9]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [17]:
#  5개의 토픽으로 지정하여 분석
temp_list = []
for blog in topic['해시태그_sub']:
    LDA_nous = okt.nouns(hangul_function(blog))
    LDA_nous = [x for x in LDA_nous if len(x) > 1 or x =='봄']  # 한글자 키워드 제거    
    LDA_nous = [word for word in LDA_nous if not word in stopwords]
    temp_list.append(LDA_nous)

dictionary = corpora.Dictionary(temp_list)
dictionary.filter_extremes(no_below=2, no_above=0.5)
corpus_test = [dictionary.doc2bow(text) for text in temp_list]

num_topics = 5
#각 학습청크에서 한번에 처리되는 문서의 수. 값이 클수록 훈련속도 up많은 메모리 필요할수도 있음.
# 값이 작을수록 더 정확한 결과를 얻을 수 있지만 훈련속도가 느려질수 있고 일반적인 값은 약 2000
chunksize = 2000 
# 학습중에 전체 말뭉치가 처리되는 횟수. 패스 수를 늘리면 더 정확한 모델을 만들 수 있지만 더 많은
# 더 많은  계산이 필요함. 일반적인 값은 약20.
passes = 20
#  각 패스내에서 모델 매개변수를 업데이트하기 위한 반복횟수. 일반적인 값은 약 400
iterations = 400
#  지정된 경우 모델 당혹도는 반복마다 평가됨. 복잡도는 모형이 보류된 테스트 세트를 얼마나 잘 예측하는지를
# 측정한 것. 이 매개변수를 설정하면 모델의 수렴을 모니터링하는데 도움이 될수있음.
#  'None'인 경우, 훈련 중에 복잡도가 계산되지 않음.
eval_every = None


temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus_test,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

lda_visualization = gensimvis.prepare(model, corpus_test, dictionary, sort_topics=True,)
pyLDAvis.save_html(lda_visualization, './result2/240122_re_group5.html')

In [18]:
top_topics = model.top_topics(corpus_test)

avg_topic_coherence = sum([t[1] for t in top_topics])/ num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -6.2351.
[([(0.14355464, '경관'),
   (0.13471946, '포토'),
   (0.06925594, '스팟'),
   (0.061317466, '자연'),
   (0.061172005, '휴식'),
   (0.059543565, '맑음'),
   (0.057195906, '커플'),
   (0.051913805, '힐링'),
   (0.03794771, '친구'),
   (0.035507612, '해변'),
   (0.030031858, '부모'),
   (0.023557302, '여름'),
   (0.021485958, '어린이'),
   (0.011794893, '치유'),
   (0.010933253, '사계절'),
   (0.010760368, '드라이브'),
   (0.010232351, '관광지'),
   (0.009518579, '청년'),
   (0.009233381, '가을'),
   (0.008607209, '봄')],
  -2.116776559828497),
 ([(0.13896804, '관광'),
   (0.12362534, '문화'),
   (0.073879525, '실내'),
   (0.06640474, '관광지'),
   (0.060782917, '유적지'),
   (0.05504319, '역사'),
   (0.055043172, '유적'),
   (0.045059316, '무장애'),
   (0.029529782, '박물관'),
   (0.023470307, '어린이'),
   (0.02167366, '테마'),
   (0.021262864, '미술'),
   (0.020637566, '사계절'),
   (0.016272865, '겨울'),
   (0.01301013, '커플'),
   (0.010859487, '마을'),
   (0.010821389, '업체'),
   (0.010175799, '맑음'),
   (0.010164745, '친구'),
   (0.

In [20]:
# 각 토픽별 20개 단어로 정하기
model.print_topics(num_words=20)

[(0,
  '0.144*"경관" + 0.135*"포토" + 0.069*"스팟" + 0.061*"자연" + 0.061*"휴식" + 0.060*"맑음" + 0.057*"커플" + 0.052*"힐링" + 0.038*"친구" + 0.036*"해변" + 0.030*"부모" + 0.024*"여름" + 0.021*"어린이" + 0.012*"치유" + 0.011*"사계절" + 0.011*"드라이브" + 0.010*"관광지" + 0.010*"청년" + 0.009*"가을" + 0.009*"봄"'),
 (1,
  '0.371*"반려동물" + 0.094*"동반" + 0.055*"관광지" + 0.042*"반입" + 0.026*"체험" + 0.025*"애견" + 0.021*"자연" + 0.017*"테마" + 0.017*"애견미용실" + 0.013*"이색" + 0.012*"스쿠버다이빙" + 0.012*"유치원" + 0.011*"어린이" + 0.010*"경관" + 0.009*"반려견" + 0.009*"해변" + 0.009*"병원" + 0.008*"무장애" + 0.008*"스파" + 0.008*"공원"'),
 (2,
  '0.275*"체험" + 0.101*"레저" + 0.068*"관광" + 0.039*"커플" + 0.039*"맑음" + 0.037*"친구" + 0.036*"어린이" + 0.029*"수상" + 0.024*"마을" + 0.022*"이색" + 0.016*"승마" + 0.015*"여름" + 0.015*"관광지" + 0.014*"해변" + 0.013*"물놀이" + 0.012*"부모" + 0.011*"산책" + 0.010*"해수욕장" + 0.010*"사계절" + 0.010*"서귀포"'),
 (3,
  '0.192*"도보" + 0.099*"자연" + 0.096*"경관" + 0.082*"걷기" + 0.078*"등산" + 0.068*"오름" + 0.054*"친구" + 0.040*"맑음" + 0.037*"커플" + 0.016*"올레" + 0.015*"러닝" + 0.015*"홀리데이인" + 0

In [21]:
df_topic = pd.DataFrame(model.print_topics(num_words=20))
df_topic.columns = ['group', 'topic']
df_topic

,group,topic
0,0,"0.144*""경관"" + 0.135*""포토"" + 0.069*""스팟"" + 0.061*""..."
1,1,"0.371*""반려동물"" + 0.094*""동반"" + 0.055*""관광지"" + 0.04..."
2,2,"0.275*""체험"" + 0.101*""레저"" + 0.068*""관광"" + 0.039*""..."
3,3,"0.192*""도보"" + 0.099*""자연"" + 0.096*""경관"" + 0.082*""..."
4,4,"0.139*""관광"" + 0.124*""문화"" + 0.074*""실내"" + 0.066*""..."


In [22]:
temp = df_topic['topic'].str.split('+', expand=True)
for i in range(20):    
    temp.rename(columns= {i: f'topic_{i}'}, inplace= True)
    temp[[f'topic_{i}_rate', f'topic_{i}_key']] = temp[f'topic_{i}'].str.split('*', expand=True)
    temp.drop(columns= f'topic_{i}', inplace=True)
    temp[f'topic_{i}_key'] = temp[f'topic_{i}_key'].apply(lambda x: x.replace('"', ''))


In [23]:
temp.reset_index(inplace=True)
temp.rename(columns={'index' : 'group'}, inplace=True)
temp

,group,topic_0_rate,topic_0_key,topic_1_rate,topic_1_key,topic_2_rate,topic_2_key,topic_3_rate,topic_3_key,topic_4_rate,...,topic_15_rate,topic_15_key,topic_16_rate,topic_16_key,topic_17_rate,topic_17_key,topic_18_rate,topic_18_key,topic_19_rate,topic_19_key
0,0,0.144,경관,0.135,포토,0.069,스팟,0.061,자연,0.061,...,0.011,드라이브,0.010,관광지,0.010,청년,0.009,가을,0.009,봄
1,1,0.371,반려동물,0.094,동반,0.055,관광지,0.042,반입,0.026,...,0.009,해변,0.009,병원,0.008,무장애,0.008,스파,0.008,공원
2,2,0.275,체험,0.101,레저,0.068,관광,0.039,커플,0.039,...,0.012,부모,0.011,산책,0.010,해수욕장,0.010,사계절,0.010,서귀포
3,3,0.192,도보,0.099,자연,0.096,경관,0.082,걷기,0.078,...,0.011,어린이,0.008,일몰,0.008,숲길,0.006,일출,0.006,단풍
4,4,0.139,관광,0.124,문화,0.074,실내,0.066,관광지,0.061,...,0.011,마을,0.011,업체,0.010,맑음,0.010,친구,0.010,아이비


In [24]:
topic_240112_1 = temp

In [25]:
topic_240112_1.to_csv('./240122_topic_group5.csv', index=False)

In [26]:
topic5 = pd.read_csv('./240122_topic_group5.csv')

,group,topic_0_rate,topic_0_key,topic_1_rate,topic_1_key,topic_2_rate,topic_2_key,topic_3_rate,topic_3_key,topic_4_rate,...,topic_15_rate,topic_15_key,topic_16_rate,topic_16_key,topic_17_rate,topic_17_key,topic_18_rate,topic_18_key,topic_19_rate,topic_19_key
0,0,0.144,경관,0.135,포토,0.069,스팟,0.061,자연,0.061,...,0.011,드라이브,0.010,관광지,0.010,청년,0.009,가을,0.009,봄
1,1,0.371,반려동물,0.094,동반,0.055,관광지,0.042,반입,0.026,...,0.009,해변,0.009,병원,0.008,무장애,0.008,스파,0.008,공원
2,2,0.275,체험,0.101,레저,0.068,관광,0.039,커플,0.039,...,0.012,부모,0.011,산책,0.010,해수욕장,0.010,사계절,0.010,서귀포
3,3,0.192,도보,0.099,자연,0.096,경관,0.082,걷기,0.078,...,0.011,어린이,0.008,일몰,0.008,숲길,0.006,일출,0.006,단풍
4,4,0.139,관광,0.124,문화,0.074,실내,0.066,관광지,0.061,...,0.011,마을,0.011,업체,0.010,맑음,0.010,친구,0.010,아이비


In [ ]:
# key 컬럼만 따로 저장하기
subset=topic5.iloc[:,2:42:2]

In [ ]:
topic5['topic'] = subset.apply(lambda row: ','.join(row.astype(str)), axis=1)

In [ ]:
new_topic5=topic5.iloc[:,-1]

In [ ]:
topic5_list=pd.DataFrame(new_topic5)
topic5_list.to_csv('./240122_concat_topic_re.csv', index=False)

In [ ]:
concat_topic = pd.read_csv('./240122_concat_topic_re.csv')

In [ ]:
#  topic 열의 공백 없애기
concat_topic['topic'] = concat_topic['topic'].str.replace(' ','')

In [ ]:
concat_topic['topic_sub'] = concat_topic['topic'].apply(lambda x: x.split(','))

In [ ]:
#  관광지 최종 데이터 가져오기
jeju = pd.read_csv('./result/240122_tag_re.csv')

In [ ]:
#  관광지 해시태그별 5개 그룹중 어디에 속하는지 분석하기
result = []
for i in range(len(jeju)):
    title = jeju['관광지명'][i]
    cnt = 0
    for group_tag in concat_topic['topic_sub']:
        cnt += 1
        print(f'{cnt}그룹')
        for tag in group_tag :
            result_g = []
            if jeju['해시태그_sub'][i].find(tag) >= 0:
                result.append({'title' : title,
                                 'group' : cnt,
                                 'tag' : tag})
            else :
                continue

In [ ]:
group_result=pd.DataFrame(result)

In [ ]:
#  그룹별 해시태그와 관광지별 해시태그 비교하여 겹치는 데이터값 찾기
test = group_result.groupby(['title', 'group'])['tag'].agg(list)
test = pd.DataFrame(test)
test.reset_index(inplace=True)
test

In [ ]:
# #  그룹별 해시태그와 관광지별 해시태그 비교하여 겹치는 데이터값  갯수 찾기
temp = group_result.groupby(['title', 'group']).count()
temp = temp.reset_index()
temp

In [ ]:
temp.columns = ['title', 'group', 'tag_count']
test['tag_count']= temp['tag_count']

각 그룹별 속한 태그의 수(합계) 컬럼 생성

In [ ]:
test_1 = test[test['group'] == 1]
test_1.columns = ['title', 'group', 'tag','group_1']
test_1.drop(columns='group', inplace= True)
test_1

In [ ]:
test_2 = test[test['group'] == 2]
test_2.columns = ['title', 'group', 'tag','group_2']
test_2.drop(columns='group', inplace= True)
test_2

In [ ]:
test_3 = test[test['group'] == 3]
test_3.columns = ['title', 'group', 'tag','group_3']
test_3.drop(columns='group', inplace= True)
test_3

In [ ]:
test_4 = test[test['group'] == 4]
test_4.columns = ['title', 'group', 'tag','group_4']
test_4.drop(columns='group', inplace= True)
test_4

In [ ]:
test_5 = test[test['group'] == 5]
test_5.columns = ['title', 'group', 'tag','group_5']
test_5.drop(columns='group', inplace= True)
test_5

In [ ]:
#  다섯개의 데이터 합치기
merge_1 =pd.merge(test_1, test_2, on='title',suffixes=('_test_1', '_test_2'))
merge_2= pd.merge(merge_1, test_3, on='title',how='outer',suffixes=('_merged', '_test_3'))
merge_3 =pd.merge(merge_2, test_4, on='title',how='outer',suffixes=('_merged', '_test_4'))
merge_all = pd.merge(merge_3, test_5, on='title', how='outer',suffixes=('_merged', '_test_5'))

In [ ]:
# 컬럼 순서 바꾸기
merge_all= merge_all[['title','group_1','tag_test_1','group_2','tag_test_2','group_3','tag_merged','group_4','tag_test_4','group_5','tag']]

In [ ]:
# 각 관광지별 태그 갯수(합계) 중 최대값을 데이터값으로 갖는 컬럼생성
merge_all['Group'] = merge_all.apply(lambda row: max(row['group_1'], row['group_2'],row['group_3'],row['group_4'],row['group_5']),axis=1)

In [ ]:
#  최대값에 해당하는 그룹 찾기
merge_all['group1'] = (merge_all['Group'] == merge_all['group_1']).astype(int)
merge_all['group2'] = (merge_all['Group'] == merge_all['group_2']).astype(int)
merge_all['group3'] = (merge_all['Group'] == merge_all['group_3']).astype(int)
merge_all['group4'] = (merge_all['Group'] == merge_all['group_4']).astype(int)
merge_all['group5'] = (merge_all['Group'] == merge_all['group_5']).astype(int)

In [ ]:
#  컬럼이름 변경
merge_all.rename(columns={'title':'관광지명','tag_test_1':'group_1_tag','tag_test_2':'group_2_tag','tag_merged':'group_3_tag','tag_test_4':'group_4_tag','tag':'group_5_tag'},inplace=True)

In [ ]:
merge_all.to_csv('./result2/240122_5group.csv', index=False)
first_group=pd.read_csv('./result2/240122_5group.csv')

In [ ]:
#  그룹별 나누기 및 저장하기
group_1=first_group.loc[first_group['group1'] == 1,['관광지명','group_1_tag','group1']]
group_2=first_group.loc[first_group['group2'] == 1,['관광지명','group_2_tag','group2']]
group_3=first_group.loc[first_group['group3'] == 1,['관광지명','group_3_tag','group3']]
group_4=first_group.loc[first_group['group4'] == 1,['관광지명','group_4_tag','group4']]
group_5=first_group.loc[first_group['group5'] == 1,['관광지명','group_5_tag','group5']]

group_1.to_csv('./result2/group1.csv', index=False)
group_2.to_csv('./result2/group2.csv', index=False)
group_3.to_csv('./result2/group3.csv', index=False)
group_4.to_csv('./result2/group4.csv', index=False)
group_5.to_csv('./result2/group5.csv', index=False)